# Creating DB and Loading Data

In [1]:
import pandas as pd
df = pd.read_csv('InsuranceData.csv')

In [2]:
df.head()

,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus
0,P1,C1,Female,73,Auto,13-02-2024,13-02-2025,240.64,33175.10,C1,NaN,0.00,Rejected
1,P2,C2,Male,44,Travel,03-03-2024,03-03-2025,1059.73,85046.42,C2,22-06-2024,1493.06,Pending
2,P3,C3,Female,28,Travel,15-06-2024,15-06-2025,1019.59,68525.53,C3,NaN,0.00,Rejected
3,P4,C4,Male,85,Travel,03-06-2024,03-06-2025,549.70,17053.27,C4,25-02-2025,5021.34,Pending
4,P5,C5,Female,57,Travel,13-06-2024,13-06-2025,841.15,55007.27,C5,30-11-2024,1347.46,Pending


In [3]:
# Creating a SQL DB.

import sqlite3
conn = sqlite3.connect('insurance_data_analysis.db')

In [4]:
# loading the data into the DB

df.to_sql('insurance', conn)

10004

# Setting up the SQL Magic

In [5]:
%load_ext sql
%sql sqlite:///insurance_data_analysis.db
%config SqlMagic.style = 'PLAIN_COLUMNS'

Connecting to 'sqlite:///insurance_data_analysis.db'

### Show the first 5 rows of the dataset.

In [6]:
%%sql

select * from insurance limit 5;

Running query in 'sqlite:///insurance_data_analysis.db'

index,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus
0,P1,C1,Female,73,Auto,13-02-2024,13-02-2025,240.64,33175.1,C1,None,0.0,Rejected
1,P2,C2,Male,44,Travel,03-03-2024,03-03-2025,1059.73,85046.42,C2,22-06-2024,1493.06,Pending
2,P3,C3,Female,28,Travel,15-06-2024,15-06-2025,1019.59,68525.53,C3,None,0.0,Rejected
3,P4,C4,Male,85,Travel,03-06-2024,03-06-2025,549.7,17053.27,C4,25-02-2025,5021.34,Pending
4,P5,C5,Female,57,Travel,13-06-2024,13-06-2025,841.15,55007.27,C5,30-11-2024,1347.46,Pending


### Find the total PremiumAmount, CoverageAmount and ClaimAmount

In [7]:
%%sql
select 
    sum(premiumamount) as 'Premium Amount',
    sum(coverageamount) as 'Coverage Amount',
    sum(claimamount) as 'Claim Amount'
from insurance;

Running query in 'sqlite:///insurance_data_analysis.db'

Premium Amount,Coverage Amount,Claim Amount
5976969.88,600551497.67,16912302.73


### Determine the parity of insurance as per gender.

In [8]:
%%sql
select gender, count(gender) as gender_count
from insurance
group by gender;

-- From this we can say that the data contains equal example of each class.

Running query in 'sqlite:///insurance_data_analysis.db'

Gender,gender_count
Female,5001
Male,5003


### Find the sum of Premium Amount by Policy Type

In [9]:
%%sql
select policytype, sum(premiumamount) as 'Premium Amount'
from insurance
group by policytype
order by sum(premiumamount) desc;

-- From below we can say that people mostly buy insurance for the Travel and Health.

Running query in 'sqlite:///insurance_data_analysis.db'

PolicyType,Premium Amount
Travel,2478329.99
Health,1199727.04
Auto,959863.74
Life,734816.05
Home,604233.06


### Find the Claim Amount by Age Group

In [10]:
%%sql
-- We will first need to create few Age Groups 
-- Young - (0 - 18)
-- Elder - (18 - 40)
-- Adult - (40 - 60)
    
ALTER TABLE insurance ADD COLUMN agegroup TEXT;

UPDATE insurance
SET agegroup = CASE
    WHEN age > 0 AND age <= 18 THEN 'Young'
    WHEN age > 18 AND age <= 40 THEN 'Elder'
    WHEN age > 40 AND age <= 60 THEN 'Adult'
    WHEN age > 60 THEN 'Senior Citizen'
    
END;

select * from insurance limit 5;

Running query in 'sqlite:///insurance_data_analysis.db'

10004 rows affected.

index,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus,agegroup
0,P1,C1,Female,73,Auto,13-02-2024,13-02-2025,240.64,33175.1,C1,None,0.0,Rejected,Senior Citizen
1,P2,C2,Male,44,Travel,03-03-2024,03-03-2025,1059.73,85046.42,C2,22-06-2024,1493.06,Pending,Adult
2,P3,C3,Female,28,Travel,15-06-2024,15-06-2025,1019.59,68525.53,C3,None,0.0,Rejected,Elder
3,P4,C4,Male,85,Travel,03-06-2024,03-06-2025,549.7,17053.27,C4,25-02-2025,5021.34,Pending,Senior Citizen
4,P5,C5,Female,57,Travel,13-06-2024,13-06-2025,841.15,55007.27,C5,30-11-2024,1347.46,Pending,Adult


In [11]:
%%sql
select agegroup, sum(claimamount) as 'Claim Amount'
from insurance
group by agegroup
order by sum(claimamount) desc;

-- Here we can observe that most people tend to claim insurance in the age when they are 60 and above(which is obvious).

Running query in 'sqlite:///insurance_data_analysis.db'

agegroup,Claim Amount
Senior Citizen,6393320.53
Elder,5235186.0
Adult,5024947.08
Young,258849.12


### Find the Number of Claims by Claim Status

In [12]:
%%sql
select claimstatus, count(claimstatus) as 'Claim Status Count'
from insurance
group by claimstatus
order by count(claimstatus) desc;

-- Every insurance companies try to deny most of the claims to maximize their profit.

Running query in 'sqlite:///insurance_data_analysis.db'

ClaimStatus,Claim Status Count
Rejected,4355
Settled,3386
Pending,2263


### Find the Top Claims Settled by Insurance Company for each Insurance Type

In [13]:
%%sql
with cte as (
    select *, row_number() over (partition by policytype
            order by claimamount desc) as rn
    from insurance
    where claimstatus='Settled'
)
    
select * from cte
where rn=1;

-- This data can help advertising team to bring more customers.

Running query in 'sqlite:///insurance_data_analysis.db'

index,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus,agegroup,rn
4339,P4340,C4340,Male,50,Auto,21-05-2024,21-05-2025,213.84,96575.23,C4340,03-11-2024,5498.58,Settled,Adult,1
7198,P7199,C7199,Female,45,Health,29-12-2023,29-12-2024,944.05,81736.24,C7199,29-05-2024,5477.51,Settled,Adult,1
2656,P2657,C2657,Female,58,Home,29-04-2024,29-04-2025,854.23,15687.58,C2657,25-01-2025,5494.7,Settled,Adult,1
9621,P9622,C9622,Male,45,Life,30-01-2024,30-01-2025,842.22,30897.41,C9622,04-08-2024,5491.77,Settled,Adult,1
3827,P3828,C3828,Male,70,Travel,23-10-2023,23-10-2024,972.96,84343.94,C3828,27-01-2024,5489.38,Settled,Senior Citizen,1


### Identify the Customers with Overlapping Insurances

In [14]:
%%sql

select *
from insurance i1
join insurance i2 
  on i1.customerid = i2.customerid
  and i1.policynumber != i2.policynumber
where i1.policystartdate < i2.policyenddate
  and i1.policyenddate > i2.policystartdate;

-- This will help identify the customrs who may have multiple insurnces and try to get claims by each.
-- But there are no such cases with overlapping insurances.

Running query in 'sqlite:///insurance_data_analysis.db'

index,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus,agegroup,index_1,PolicyNumber_1,CustomerID_1,Gender_1,Age_1,PolicyType_1,PolicyStartDate_1,PolicyEndDate_1,PremiumAmount_1,CoverageAmount_1,ClaimNumber_1,ClaimDate_1,ClaimAmount_1,ClaimStatus_1,agegroup_1


### Find the count of Monthly Claims

In [15]:
%%sql

with cte as (
    select substr(claimdate, 4, 2) as claimmonth
    from insurance
    where claimstatus='Settled'
)

select claimmonth, count(claimmonth) as 'Number of Claims'
from cte
group by claimmonth
order by count(*) desc;

-- Here even though we cannot identify any trend but it is good to have this figures for monthly analysis report.

Running query in 'sqlite:///insurance_data_analysis.db'

claimmonth,Number of Claims
01,304
09,299
07,299
12,294
05,293
03,292
11,279
10,274
04,272
02,272


### Find Insurances who never made any Claims

In [16]:
%%sql

select * from insurance
where claimamount=0
limit 5;

-- These can be used at times to reward customers for not making any claims and provide them some extra benefits to retain them.
-- This is a very crucial element for any insurance company to retain its customers.

Running query in 'sqlite:///insurance_data_analysis.db'

index,PolicyNumber,CustomerID,Gender,Age,PolicyType,PolicyStartDate,PolicyEndDate,PremiumAmount,CoverageAmount,ClaimNumber,ClaimDate,ClaimAmount,ClaimStatus,agegroup
0,P1,C1,Female,73,Auto,13-02-2024,13-02-2025,240.64,33175.1,C1,None,0.0,Rejected,Senior Citizen
2,P3,C3,Female,28,Travel,15-06-2024,15-06-2025,1019.59,68525.53,C3,None,0.0,Rejected,Elder
10,P11,C11,Female,65,Health,04-08-2023,04-08-2024,333.2,46868.19,C11,None,0.0,Rejected,Senior Citizen
11,P12,C12,Male,47,Home,16-09-2023,16-09-2024,645.73,67012.84,C12,None,0.0,Rejected,Adult
12,P13,C13,Female,46,Travel,17-08-2023,17-08-2024,646.14,87450.81,C13,None,0.0,Rejected,Adult


### Calculate Coverage-to-Premium Ratio Ranking

In [17]:
%%sql

with ratio_cte as (
  select
    policynumber,
    policytype,
    customerid,
    coverageamount,
    premiumamount,
    cast(coverageamount as float) / nullif(premiumamount, 0) as ratio
  from insurance
)

select *,
  dense_rank() over (order by ratio desc) as ratio_rank
from ratio_cte;

-- Bloated Coverage Amounts helps attract more customers.

Running query in 'sqlite:///insurance_data_analysis.db'

policynumber,policytype,customerid,coverageamount,premiumamount,ratio,ratio_rank
P338,Auto,C338,108138.56,102.23,1057.7967328572824,1
P9963,Travel,C9963,109921.43,106.47,1032.4169249553865,2
P7022,Travel,C7022,104854.97,102.14,1026.5808693949482,3
P889,Travel,C889,109407.46,107.35,1019.1659059152307,4
P8554,Travel,C8554,108094.38,106.21,1017.7420205253744,5
P8371,Health,C8371,106888.37,106.41,1004.495536133822,6
P9658,Life,C9658,107002.0,106.68,1003.018372703412,7
P9916,Travel,C9916,106228.25,106.61,996.4191914454555,8
P9798,Home,C9798,105640.68,106.49,992.0244154380692,9
P1339,Health,C1339,109450.99,110.55,990.0587064676618,10


### Calculate the Market Share in each Policy Type

In [18]:
%%sql

with type_counts as (
  select policytype, count(*) as type_total
  from insurance
  group by policytype
),

total_count as (
  select count(*) as total_policies
  from insurance
)

select 
  t.policytype,
  t.type_total,
  round((t.type_total * 100.0) / tc.total_policies, 2) as market_share_percent
from type_counts t
cross join total_count tc
order by market_share_percent desc;

-- This will give us a better idea of the market segment that are supposed to be targeted.

Running query in 'sqlite:///insurance_data_analysis.db'

policytype,type_total,market_share_percent
Travel,4148,41.46
Health,2000,19.99
Auto,1595,15.94
Life,1248,12.48
Home,1013,10.13
